# Load corresponding input file

In [1]:
import cPickle as pickle
import tensorflow as tf

In [ ]:
# Pickle:
f = open('word-equal-context.p', 'rb')
info = pickle.load(f)
word_pruned_dictionary = info["word_pruned_dictionary"]
context_pruned_dictionary = info["context_pruned_dictionary"]
del info
f.close() 

In [9]:
word_pruned_dictionary

{'writings': 4881,
 'concessions': 4839,
 'yellow': 1028,
 'four': 3117,
 'diamondbacks': 1737,
 'woods': 1,
 'hanging': 7718,
 'woody': 2,
 'cyprus': 2040,
 'aggression': 2041,
 'papandreou': 9834,
 'increase': 10191,
 'granting': 3032,
 'eligible': 6769,
 'electricity': 4882,
 'unanswered': 4883,
 'lord': 2053,
 'meadows': 5825,
 'sinking': 2043,
 'hormone': 10897,
 'kent': 1032,
 'retail': 5574,
 'regional': 1030,
 'dell': 12807,
 'foul': 2038,
 'politician': 699,
 'stabbed': 5826,
 'bringing': 3,
 'markers': 11841,
 'wooded': 4,
 'basics': 2045,
 'internally': 3034,
 'prize': 4884,
 'obstruction': 7210,
 'wooden': 6,
 'wednesday': 7,
 'solid': 5827,
 'jihad': 6768,
 'shouting': 9500,
 'succession': 13766,
 'provided': 5695,
 'path': 9578,
 'commented': 9836,
 'bitterly': 5772,
 'annie': 2962,
 'charter': 12808,
 'distributors': 10468,
 'specially': 4167,
 'tired': 3035,
 'miller': 5849,
 'mirage': 8759,
 'bacon': 10899,
 'pulse': 3036,
 'budget': 7893,
 '270': 8,
 'elegant': 2069,


# Create Graph of the Neural Network

# Run the Neural Network

In [ ]:
def one_hot_encoding(labels, num_labels):
    return (np.arange(num_labels) == labels[:,None]).astype(np.float32)

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

In [ ]:
# Dimensions of the embedding
embedding_size = 500

# Size of the batch to be processed in each step
batch_size = 128

# Number of iterations
num_steps = 3001 

# Length of the context dictionary
num_labels = 1000 

# Length of the words dictionary
num_features = 1000 

    graph = tf.Graph()
    with graph.as_default():
        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, num_features))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)
        
        # TODO: Generate values with a -1 to 1 range
        embedding = tf.Variable(tf.truncated_normal([num_features, embedding_size]))
        bias = tf.Variable(tf.zeros([num_labels]))
        
        # TODO: Create inner layers.
        
        # Traditional Softmax to compute the loss
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
        
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
        
        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(logits)

        # Predictions for the training, validation, and test data.
        valid_prediction = tf.nn.softmax(logits)
        
        #valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
        test_prediction = tf.nn.softmax(tf_test_dataset)

In [ ]:
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size)]
            batch_labels = train_labels[offset:(offset + batch_size)]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                #print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))